In [5]:


import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from tqdm import tqdm
import os


In [3]:

model = YOLO("best.pt")  


In [4]:
# Confirm model class labels
print(model.names)


{0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}


In [8]:
video_path = '15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# Get video details
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Output writer setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_reid_3.mp4', fourcc, fps, (width, height))

# Initialize DeepSort
tracker = DeepSort(max_age=30, n_init=2) 

In [ ]:
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    print(f"[INFO] Processing frame: {frame_idx}")
    
    # Run YOLOv8 inference
    results = model(frame)[0]  # [0] to get first result

    detections = []
    for box in results.boxes:
        cls = int(box.cls.item())
        conf = float(box.conf.item())
        
        # Only player (class 2), confidence threshold 0.5+
        if cls == 2 and conf > 0.5:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            w, h = x2 - x1, y2 - y1

            # Optional filter: ignore tiny detections (false positives)
            if w < 20 or h < 40:
                continue

            detections.append(([x1, y1, w, h], conf, 'player'))


    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.track_id is None:
            continue

        # Get track box
        x1, y1, x2, y2 = map(int, track.to_ltrb())

        # Filter out giant or tiny fake boxes
        w, h = x2 - x1, y2 - y1
        if w < 20 or h < 40 or w > 400 or h > 600:
            continue

        # Draw bounding box & ID label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
        cv2.putText(frame, f'Player {track.track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)



    # Write the output frame
    out.write(frame)

    # Optional: Show live preview (can comment out)
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_idx += 1

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("[DONE] Output saved to: output_reid.mp4")

[INFO] Processing frame: 0



0: 384x640 1 ball, 16 players, 2 referees, 346.5ms
Speed: 3.4ms preprocess, 346.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 1

0: 384x640 18 players, 2 referees, 258.5ms
Speed: 0.9ms preprocess, 258.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 2

0: 384x640 1 ball, 16 players, 2 referees, 258.0ms
Speed: 1.0ms preprocess, 258.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 3

0: 384x640 1 ball, 14 players, 2 referees, 266.2ms
Speed: 0.8ms preprocess, 266.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 4

0: 384x640 1 ball, 14 players, 2 referees, 261.0ms
Speed: 1.0ms preprocess, 261.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Processing frame: 5

0: 384x640 1 ball, 16 players, 2 referees, 252.3ms
Speed: 1.0ms preprocess, 252.3ms inference, 0.6ms postprocess per image 

: 

NameError: name 'model' is not defined